In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs

In [ ]:
dpsi0 = hs.load("Data/CuTwinTilt/20220305_150329/SPED_64x64x12_50x50_scanRot340_NBD_alpha5_spot0p5nm_CL12cm_1deg_100Hz_20ms_substrate.hspy")
dpsi1 = hs.load("Data/CuTwinTilt/20220305_153533/SPED_64x64x12_50x50_scanRot340_NBD_alpha5_spot0p5nm_CL12cm_1deg_100Hz_20ms.hspy")
dpsi2 = hs.load("Data/CuTwinTilt/20220305_155526/SPED_64x64x12_50x50_scanRot340_NBD_alpha5_spot0p5nm_CL12cm_1deg_100Hz_20ms.hspy")
dpsi3 = hs.load("Data/CuTwinTilt/20220305_162230/SPED_64x64x12_50x50_scanRot340_NBD_alpha5_spot0p5nm_CL12cm_1deg_100Hz_20ms.hspy")
dpsi4 = hs.load("Data/CuTwinTilt/20220305_164519/SPED_64x64x12_50x50_scanRot340_NBD_alpha5_spot0p5nm_CL12cm_1deg_100Hz_20ms.hspy")

In [ ]:
dpsi0 = dpsi0.inav[20:64,40:64]
dpsi1 = dpsi1.inav[20:64,40:64]
dpsi2 = dpsi2.inav[20:64,40:64]
dpsi3 = dpsi3.inav[20:64,40:64]
dpsi4 = dpsi4.inav[20:64,40:64]

In [ ]:
sdpsi0 = dpsi0.mean(axis = (0,1))
sdpsi1 = dpsi1.mean(axis = (0,1))
sdpsi2 = dpsi2.mean(axis = (0,1))
sdpsi3 = dpsi3.mean(axis = (0,1))
sdpsi4 = dpsi4.mean(axis = (0,1))

In [ ]:
from skimage import filters

def crop_minimum(image, minimum=3):
    copied = image.copy()
    copied[copied <= minimum] = 0.
    return copied

In [ ]:
procdat = sdpsi0.subtract_diffraction_background(method="difference of gaussians",
                                                            min_sigma=4,
                                                            max_sigma=12, 
                                                            lazy_result=False)
procdat = procdat.map(filters.gaussian, sigma=1, inplace=False)
procdat = procdat.map(crop_minimum, minimum = 1.3, inplace=False)

In [ ]:
procdat.plot(vmax = 30)

In [ ]:
alldpsi = hs.signals.Signal2D((sdpsi0.data,
                               sdpsi1.data,
                               sdpsi2.data,
                               sdpsi3.data,
                               sdpsi4.data,
                              ))

In [ ]:
alldpsi.plot(vmax = 50)

In [ ]:
alldpsi.set_signal_type('electron_diffraction')

In [ ]:
procdat = alldpsi.subtract_diffraction_background(method="difference of gaussians",
                                                            min_sigma=6,
                                                            max_sigma=7, 
                                                            lazy_result=False)
procdat = procdat.map(filters.gaussian, sigma=1, inplace=False)
procdat = procdat.map(crop_minimum, minimum = 1.3, inplace=False)

In [ ]:
procdat.plot(vmax = 30)

In [ ]:
alldpsi = procdat

In [ ]:
alldpsi.set_signal_type('electron_diffraction')

In [ ]:
%%capture burn
import diffpy
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from pyxem.utils.calibration_utils import find_diffraction_calibration

def template_log_func(x):
    return np.log10(x + 0.01)

resolution = 1 
grid = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")
structure_matrix = diffpy.structure.loadStructure("Data/Si_mp-149_conventional_standard.cif")

diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)
library_phases = StructureLibrary(["phase"], [structure_matrix], [grid])

found_cal, corrline, found_cals = find_diffraction_calibration(alldpsi,0.0095,library_phases,lib_gen,20,
                                                               max_excitation_error= 0.08,
                                                               intensity_transform_function=template_log_func,
                                                               normalize_images = True,
                                                               delta_r = 0.5,
                                                               max_r = np.sqrt(128**2+128**2) * 0.90
                                                              )

In [ ]:
found_cal

In [ ]:
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
resolution = 0.5 
grid_cub = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")
print("Number of patterns: ", grid_cub.shape[0])

In [ ]:
import diffpy
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator

In [ ]:
diffraction_calibration = 0.009381971999999999
half_shape = (alldpsi.data.shape[-2]//2, alldpsi.data.shape[-1]//2)
reciprocal_radius = np.sqrt(half_shape[0]**2 + half_shape[1]**2)*diffraction_calibration

In [ ]:
structure_matrix = diffpy.structure.loadStructure("Data/Si_mp-149_conventional_standard.cif")
# structure_matrix = diffpy.structure.loadStructure("Data/Si_sudofcc.cif")

diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)

library_phases = StructureLibrary(["Si"], [structure_matrix], [grid_cub])

diff_lib = lib_gen.get_diffraction_library(library_phases,
                                           calibration=diffraction_calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=half_shape,
                                           with_direct_beam=False,
                                           max_excitation_error=0.06)

In [ ]:
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls
from pyxem.utils import polar_transform_utils as ptutls
from pyxem.utils import expt_utils as eutls

In [ ]:
def template_log_func(x):
    return np.log10(x + 0.01)

In [ ]:
frac_keep = 1
n_best = 30
n_keep = None

delta_r = 0.5
delta_theta = 0.5
max_r = None
intensity_transform_function = template_log_func
direct_beam_position = None
normalize_image = True
normalize_templates = True

result, phasedict = iutls.index_dataset_with_template_rotation(alldpsi,
                                                    diff_lib,
                                                    phases = ["Si"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=intensity_transform_function,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='gpu'
                                                    )

In [ ]:
from orix import plot
from orix.crystal_map import CrystalMap, Phase, PhaseList
from orix.io import load, save
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Vector3d, Miller

In [ ]:
from pyxem.signals.indexation_results import results_dict_to_crystal_map

In [ ]:
xmap = results_dict_to_crystal_map(result,phasedict)

In [ ]:
rori = xmap.orientations.map_into_symmetry_reduced_zone()

In [ ]:
xmap.phases[0].space_group = 227

In [ ]:
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Si"].point_group, direction=Vector3d.zvector())
rgb_fe = ckey_m3m.orientation2color(xmap["Si"].orientations)

In [ ]:
save(
    filename="Data/CuTwinTiltSixmapBGRED.h5",
    object2write=xmap,
    overwrite=False,  # Default is False
)

In [ ]:
n= np.linspace(0,4,num = 5)
n

In [ ]:
xmap = load("Data/CuTwinTiltSixmapBGRED.h5") 

In [ ]:
Miller((1,1,1))

In [ ]:
xmap.orientations.scatter(projection= 'ipf', direction = Vector3d((0,0,1)),c = n, cmap = 'viridis')

In [ ]:
xmap.orientations.scatter(projection= 'ipf', direction = Vector3d(((1, 0, 0),(0,1,0),(0,0,1))),c = n, cmap = 'viridis')

In [ ]:
m111 = (Miller((1,1,1),  phase=Phase(point_group="m-3m")).symmetrise(unique = True))

In [ ]:
rori

In [ ]:
things = ((rori.conj).outer(Miller((1,0,1))))

In [ ]:
things = ((rori.conj).outer(m111))

In [ ]:
things.scatter(hemisphere='upper', grid = True, grid_resolution= (5,5), c = n)

In [ ]:
Orientation(xmap.rotations[0:3,0:10], symmetry=symmetry.Oh).scatter(projection= 'ipf', direction = Vector3d(((1, 0, 0),(0,1,0),(0,0,1))),c = n, cmap = 'viridis')

In [ ]:
xmap.orientations.scatter(projection= 'axangle', cmap = 'viridis', c = n)

In [ ]:
Orientation(xmap.rotations[0:3,0:10], symmetry=symmetry.Oh)

In [ ]:
print(np.degrees((xmap.orientations[0] - xmap.orientations[1]).angle.data))
print(np.degrees((xmap.orientations[1] - xmap.orientations[2]).angle.data))
print(np.degrees((xmap.orientations[0] - xmap.orientations[2]).angle.data))
print(np.degrees((xmap.orientations[0] - xmap.orientations[3]).angle.data))
print(np.degrees((xmap.orientations[3] - xmap.orientations[4]).angle.data))
print(np.degrees((xmap.orientations[0] - xmap.orientations[4]).angle.data))

In [ ]:
xmap.orientations[0] - xmap.orientations[2]

In [ ]:
from diffpy.structure import Atom, Lattice, Structure
p = Phase(
    name="si",
    space_group=227,
    structure=Structure(
        atoms=[Atom("si", [0, 0, 0])],
        lattice=Lattice(0.546, 0.546, 0.546, 90, 90, 90)
    ))

In [ ]:
w.cross(Miller((1,1,1)))

In [ ]:
ori0 = xmap.orientations[0]
ori2 = xmap.orientations[2]
w = ori0.outer(ori2).axis

In [ ]:
tiltx = (xmap.orientations[0] - xmap.orientations[2]).axis
tilty = (xmap.orientations[0] - xmap.orientations[4]).axis

In [ ]:
# v6 = Vector3d.from_polar(azimuth=np.deg2rad([0, 60]), polar=np.deg2rad([75, 60]))
v6 = w
colors1 = ["c", "m"]
colors2 = ["C0", "C1","C2","C3","C4"]
fig6 = things.scatter(
    axes_labels=["RD", "TD", None],
    hemisphere="upper",
    c=colors2,
    s=100,
    grid = True, 
    grid_resolution= (5,5),
    show_hemisphere_label=True,
    return_figure=True
)
v6.draw_circle(color=colors1, linewidth=2, figure=fig6)
v6.scatter(c = colors1, figure = fig6)

In [ ]:
newbest = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]),np.int16)

for k in range(20):
    newresult= np.zeros((1,5,3))
    for i in range(result['orientation'].shape[0]):
        for j in range(result['orientation'].shape[1]):
            newresult[i,j] = result['orientation'][i,j,newbest[i,j],:]

    misoris = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]-1))

    oris = Orientation.from_euler(np.radians(newresult[:,:,:]),symmetry=symmetry.Oh)
    mis = (oris[:,0] - oris[:,1])
    misoris[:,0] = np.degrees((mis).angle.data)
    mis = (oris[:,1] - oris[:,2])
    misoris[:,1] = np.degrees((mis).angle.data)
    mis = (oris[:,0] - oris[:,3])
    misoris[:,2] = np.degrees((mis).angle.data)
    mis = (oris[:,3] - oris[:,4])
    misoris[:,3] = np.degrees((mis).angle.data)
    
    misoris[:,3]

In [ ]:
Orientation.from_euler(np.radians(newresult[:,:,:]),symmetry=symmetry.Oh)[:]

In [ ]:
misoris[:,3]

In [ ]:
np.where(np.abs(misoris[:,:]-5) > 1)[1]

In [ ]:
np.abs(misoris[:,:]-5)

In [ ]:
oris = Orientation.from_euler(np.radians(result['orientation'][:,:,:,:]),symmetry=symmetry.Oh)

In [ ]:
oris